In [1]:
import optimization as opt
import stock_work as sw
import pickle

In [2]:
sp500_tickers = sw.get_sp500_tickers()

In [3]:
try:
    with open("data.pkl", "rb") as f:
        data = pickle.load(f)

    stocks = data["stocks"]
    factors = data["factors"]

except:
    stocks, _ = sw.get_data(sp500_tickers)
    factors, _ = sw.get_factors()

with open("data.pkl", "wb") as f:
    pickle.dump({"stocks": stocks, "factors": factors}, f)

alphas, betas, idios, cov = sw.regression_results(stocks, factors)
target_betas = [.01, -.01, .01, -.01, .01, -.01, .01, -.01, .01, -.01, .01]

print(alphas.shape, betas.shape, idios.shape, cov.shape)

/Users/dipalshah/Desktop/optimized_vector_add/stock_work.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_list, period = "max", group_by = "ticker")
[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 1926-10-03 -> 2025-09-08)')
/Users/dipalshah/Desktop/optimized_vector_add/stock_work.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_list, period = "max", group_by = "ticker")
[*********************100%***********************]  11 of 11 completed


(501,) (11, 501) (501, 501) (11, 11)


In [4]:
betas.T

array([[-0.02490283,  0.18747551,  0.89380118, ...,  0.25283759,
         0.01098732, -0.06096393],
       [ 0.11704061, -0.07903348, -0.23228917, ..., -0.43634674,
         0.90760748,  0.30929299],
       [-0.14061223, -0.02389849, -0.09127643, ..., -0.18084478,
         0.31852325,  0.13935693],
       ...,
       [-0.14649939,  0.2071254 ,  0.13124629, ...,  0.32050706,
        -0.13014044,  0.62544539],
       [ 0.08745059,  0.09712324, -0.02981003, ...,  0.3533477 ,
        -0.1634047 ,  0.31893856],
       [-0.10529754,  0.23770552,  0.3271195 , ...,  0.1530496 ,
        -0.13235385,  0.12964044]])

In [5]:
vector_sum, betas, error, variance, iters = opt.optimize_constrainted_vector_sum(betas.T, target_betas, cov, idios, variance_bias = 0)

In [6]:
print("vector_sum:   ", vector_sum)
print("final_vector: ", betas)
print("error:        ", error)
print("variance:     ", variance)
print("iterations:   ", iters)

vector_sum:    [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -4.06538879e-03  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -4.78620187e-04  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -1.97129948e-01  1.72404448e-04 -2.31589014e-04
  0.000000